## More data formats

In [2]:
import os
from osgeo import ogr
import osgeopy as gp

data_dir = r'osgeopy-data'

In [2]:
# A function to print out the layers in a data source.
def print_layers(fn):
    ds = ogr.Open(fn, 0)
    if ds is None:
        raise OSError('Could not open {}'.format(fn))
    for i in range(ds.GetLayerCount()):
        lyr = ds.GetLayer(i)
        print(f'{i}: {lyr.GetName()}')

In [3]:
# Try out the function.
fn = os.path.join(data_dir, 'Washington')
print_layers(fn)

0: large_cities


### Esri file geodatabase

In [4]:
# Print out layers in an Esri file geodatabase.
fn = os.path.join(data_dir, 'global', 'natural_earth.gdb')
gp.print_layers(fn)

0: countries_10m (MultiPolygon)
1: populated_places_10m (Point)
2: countries_110m (MultiPolygon)
3: populated_places_110m (Point)


In [5]:
# Get a layer inside a feature dataset.
ds = ogr.Open(fn)
lyr = ds.GetLayer('countries_10m')

In [6]:
# Print out some attributes to make sure it worked.
gp.print_attributes(lyr, 5, ['NAME', 'POP_EST'])

FID    Geometry        NAME           POP_EST       
1      MULTIPOLYGON    Aruba          103065.0      
2      MULTIPOLYGON    Afghanistan    28400000.0    
3      MULTIPOLYGON    Angola         12799293.0    
4      MULTIPOLYGON    Anguilla       14436.0       
5      MULTIPOLYGON    Albania        3639453.0     
5 of 255 features


In [7]:
# Export a feature class from geodatabase to a shapefile.
out_folder = os.path.join(data_dir, 'global')
gdb_ds = ogr.Open(fn)
gdb_lyr = gdb_ds.GetLayerByName('countries_110m')
shp_ds = ogr.Open(out_folder, 1)
shp_ds.CopyLayer(gdb_lyr, 'countries_110m')
del shp_ds, gdb_ds

In [8]:
ds = ogr.Open(os.path.join(data_dir, 'global', 'countries_110m.shp'))
lyr = ds.GetLayer(0)

num_features = lyr.GetFeatureCount()
num_features

177

## Web Feature Service

In [28]:
# Print out layers in a WFS.
url = 'WFS:http://ogc.bgs.ac.uk/digmap625k_gsml_insp_gs/wfs?'
#gp.print_layers(url)
ds = ogr.Open(url)

In [35]:
# Get the first warning from the WFS. This might take a while because it has
# to download all of the data first.
ds = ogr.Open(url)
lyr = ds.GetLayer(1)
print(lyr.GetFeatureCount())
feat = lyr.GetNextFeature()
#gp.print_attributes(lyr,1)
print(feat.GetField('geologicUnitType'))

11242
lithodemic unit


In [37]:
# Get the first warning from the WFS by limiting the returned features to 1.
ds = ogr.Open(url + '?MAXFEATURES=1')
lyr = ds.GetLayer(1)
print(lyr.GetFeatureCount())

AttributeError: 'NoneType' object has no attribute 'GetLayer'

## Attribute filters

In [9]:
# Get the countries shapefile layer
ds = ogr.Open(os.path.join(data_dir, 'global'))
lyr = ds.GetLayer('ne_50m_admin_0_countries')

In [10]:
for i in range(lyr.GetFeatureCount()):
    feature = lyr.GetNextFeature()
    field_value = feature.GetField('continent')
    print(f'{field_value}')

North America
Asia
Africa
North America
Europe
Europe
Europe
Asia
South America
Asia
Oceania
Antarctica
Oceania
Seven seas (open ocean)
North America
Oceania
Europe
Asia
Africa
Europe
Africa
Africa
Asia
Europe
Asia
North America
Europe
North America
Europe
North America
North America
South America
South America
North America
Asia
Asia
Africa
Africa
North America
Europe
South America
Asia
Africa
Africa
Africa
Africa
Oceania
South America
Africa
Africa
North America
North America
North America
North America
Asia
Asia
Europe
Europe
Africa
North America
Europe
North America
Africa
South America
Africa
Africa
Europe
Europe
Africa
Europe
Oceania
South America
Europe
Europe
Oceania
Africa
Europe
Asia
Europe
Africa
Africa
Africa
Africa
Africa
Europe
North America
North America
North America
Oceania
South America
Asia
Seven seas (open ocean)
North America
Europe
North America
Europe
Asia
Europe
Asia
Asia
Seven seas (open ocean)
Europe
Asia
Asia
Europe
Asia
Europe
North America
Europe
Asia
Asia


In [14]:
# Apply a filter that finds countries in Asia and see how many records there
# are now.
lyr.SetAttributeFilter("continent = \'Asia\'")
lyr.GetFeatureCount()

53

In [16]:
# You can still get a feature that is not in Asia by using its FID.
lyr.GetFeature(2).GetField('name')

'Angola'

In [18]:
# Set a new filter that selects South American countries and show the results
# in blue. The old filter is no longer in effect.
lyr.SetAttributeFilter("continent = \'South America\'")
lyr.GetFeatureCount()

13

In [19]:
# Clear all attribute filters.
lyr.SetAttributeFilter(None)
lyr.GetFeatureCount()

241

## Spatial filters

In [4]:
# Get the Germany polygon.
ds = ogr.Open(os.path.join(data_dir, 'global'))
country_lyr = ds.GetLayer('ne_50m_admin_0_countries')

country_lyr.SetAttributeFilter("name = \'Germany\'")
country_lyr.GetFeatureCount()

1

In [5]:
feat = country_lyr.GetNextFeature()
germany = feat.geometry().Clone()

In [6]:
city_lyr = ds.GetLayer('ne_50m_populated_places')
city_lyr.GetFeatureCount()
# Use the Germany polygon to set a spatial filter and draw the result as blue
# circles.
city_lyr.SetSpatialFilter(germany)
city_lyr.GetFeatureCount()

5

In [ ]:
# Add an attribute filter to find the cities with a population over 1,000,000
# and draw them as red squares. Since the spatial filter is still in effect,
# you should only get large cities in Germany.
city_lyr.SetAttributeFilter('pop_min > 1000000')
city_lyr.GetFeatureCount()

In [ ]:
# Remove the spatial filter so now you get global cities with a population
# over 1,000,000. Draw the results as magenta triangles.
city_lyr.SetSpatialFilter(None)
city_lyr.GetFeatureCount()

### To clone or not to clone? 

In [ ]:
# Get a sample layer and the first feature from it.
ds = ogr.Open(os.path.join(data_dir, 'global'))
lyr = ds.GetLayer('ne_50m_admin_0_countries')
feat = lyr.GetNextFeature()

# Now get the feature's geometry and also a clone of that geometry.
geom = feat.geometry()
geom_clone = feat.geometry().Clone()


In [ ]:
# Set the feat variable to a new feature, so the original one is no longer
# accessible.
feat = lyr.GetNextFeature()

# Try to get the area of the cloned polygon. This should work just fine.
print(geom_clone.GetArea())

In [ ]:
# Try to get the area of the original polygon. This should cause Python to
# crash because the polygon is linked to the feature that is no longer
# available.
print(geom.GetArea())